In [199]:
import os
import numpy as np
import pandas as pd
import qiime2 as q2
from biom import Table
from skbio import TreeNode
from biom import load_table

def subset_match(table, metadata):
    
    """
    This function matches table & metdata
    subsets based on the shared samples.
    """
    
    # match samples
    subset = set(table.ids()) & set(metadata.index)
    # filter samples
    table = table.filter(subset)
    # reindex metadata
    metadata = metadata.reindex(table.ids())
    # make sure no zero sum features
    keep = table.ids('observation')[table.sum('observation') > 0]
    table = table.filter(keep, axis='observation')
    # make QIIME2 type
    table = q2.Artifact.import_data('FeatureTable[Frequency]',
                                    table)
    metadata = q2.Metadata(metadata)
    # return to save
    return table, metadata
    

In [207]:
# load tree
q2tree = q2.Artifact.import_data('Phylogeny[Rooted]',
                                 '../data/16S/83714_insertion_tree.relabelled.tre')
tree = q2tree.view(TreeNode)
# get table
bt = load_table('../data/16S/83714_27262_analysis_16S_DeblurReferencephylogenyforSEPPGreengenes138BIOMreferencehitbiomTrimminglength150_insertion_filter.biom')
seqs_ = [node.name for node in tree.tips()]
shared_ = list(set(seqs_) & set(bt.ids('observation')))
bt = bt.filter(shared_ , axis='observation')
# get mf
qiita_drop = ['dna_extracted','physical_specimen_remaining']
qiita_rename = {'time_numberical':'time_numerical'}
mf = pd.read_csv('../data/16S/27262_27262_analysis_mapping.txt',
                 sep='\t', index_col=0).drop(qiita_drop, axis=1)
mf = mf[mf.title.isin(['SAF_cleanroom', 'JPL_Project2'])]
mf = mf.rename(qiita_rename, axis=1)
# ensure matched 
shared_ = set(mf.index) & set(bt.ids())
mf = mf.reindex(shared_)
bt = bt.filter(shared_)
# make sure no zero sum features
keep = bt.ids('observation')[bt.sum('observation') > 0]
bt = bt.filter(keep, axis='observation')
# check
if len(set(bt.ids()) - set(mf.index)) != 0:
    raise ValueError('Some samples in metadata not in table')
if len(set(mf.index) - set(bt.ids())) != 0:
    raise ValueError('Some samples in table not in metadata')
# add x/y metdata
location_x_y = pd.read_csv('../data/16S/location-x-y-z-map.csv',
                           index_col='Location_new_SAF')
mf['jpl_x'] = [location_x_y.loc[v,'x']
               if v in location_x_y.index else np.nan
               for v in mf.jpl_location_area ]
mf['jpl_y'] = [location_x_y.loc[v,'y']
               if v in location_x_y.index else np.nan
               for v in mf.jpl_location_area ]
# import all
q2bt = q2.Artifact.import_data('FeatureTable[Frequency]', bt)
q2mf = q2.Metadata(mf)
# save
q2bt.save('../data/16S/all-table.qza')
q2mf.save('../data/16S/all-metadata.qza')
q2tree.save('../data/16S/insertion-tree.qza')
bt


34737 x 410 <class 'biom.table.Table'> with 74409 nonzero entries (0% dense)

In [208]:
# jpl2 data only
mf = mf[mf.qiita_study_id.isin(['10849'])]
# get subset
q2bt, q2mf = subset_match(bt.copy(), mf.copy())
# save
q2bt.save('../data/16S/10849-only-table.qza')
q2mf.save('../data/16S/10849-only-metadata.qza')
bt = q2bt.view(Table)
bt

31183 x 230 <class 'biom.table.Table'> with 63703 nonzero entries (0% dense)

In [132]:
# build the set of all rep-seqs
seqs_ = q2bt.view(Table).ids('observation')
seqs_ = '\n'.join(['>'+i+'\n'+i for i in seqs_])
f = open("../data/16S/rep-seqs.fa", "w")
f.write(seqs_)
f.close()

In [133]:
# import the rep-seqs
!qiime tools import \
    --input-path ../data/16S/rep-seqs.fa\
    --output-path ../data/16S/rep-seqs.qza\
    --type 'FeatureData[Sequence]'

Imported ../data/16S/rep-seqs.fa as DNASequencesDirectoryFormat to ../data/16S/rep-seqs.qza


In [134]:
# run taxonomic classification (run on cluster - big compute step)
!qiime feature-classifier classify-sklearn \
  --i-classifier ../data/16S/gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads ../data/16S/rep-seqs.qza \
  --o-classification ../data/16S/taxonomy.qza


Saved FeatureData[Taxonomy] to: ../data/16S/taxonomy.qza


In [165]:
q2tax = q2.Artifact.load('../data/16S/taxonomy.qza')
q2tax = q2tax.view(q2.Metadata)
taxdf = q2tax.to_dataframe()
taxdf_split = taxdf.Taxon.str.split(';', expand=True)
taxdf_split.columns = ['kingdom','phylum','class',
                       'order','family','genus','sp.']
taxdf = pd.concat([taxdf, taxdf_split], axis=1)
q2tax = q2.Metadata(taxdf)
q2tax.save('../data/16S/all-taxonomy.qza')


In [209]:
# create matched data splits 

path = '../data/16S/data-subsets'
pma_map = {'TRUE':'pma-treatment',
           'FALSE':'no-pma-treatment'}
cntrl_map = {'0':'rooms-only',
             '1':'controls-only'}

# pma and non-pma treated
for pma_type, pma_df in mf.groupby('jpl_pma'):
    # get subset
    pma_tbl_subset, pma_mf_subset = subset_match(bt.copy(),
                                                 pma_df.copy())
    # save
    name_pma = pma_map[pma_type]
    pma_tbl_subset.save(os.path.join(path, '-'.join([name_pma, 'table.qza'])))
    pma_mf_subset.save(os.path.join(path, '-'.join([name_pma, 'metadata.qza'])))

    # control and non-control
    for cntrl_type, cntrl_df in pma_df.groupby('jpl_controltype_0_1'):
        # get subset
        cntrl_tbl_subset, cntrl_mf_subset = subset_match(bt.copy(),
                                                         cntrl_df.copy())   
        # save
        name_cntrl = cntrl_map[cntrl_type]
        cntrl_tbl_subset.save(os.path.join(path, '-'.join([name_pma,
                                                           name_cntrl,
                                                           'table.qza'])))
        cntrl_mf_subset.save(os.path.join(path, '-'.join([name_pma,
                                                          name_cntrl,
                                                          'metadata.qza'])))

        # by time-ordered
        for t_type, t_df in cntrl_df.groupby('time_numerical'):
            # get subset
            t_tbl_subset, t_mf_subset = subset_match(bt.copy(),
                                                     t_df.copy())   
            # save
            name_cntrl = cntrl_map[cntrl_type]
            t_tbl_subset.save(os.path.join(path, '-'.join([name_pma,
                                                           name_cntrl,
                                                           t_type,
                                                           'table.qza'])))
            t_mf_subset.save(os.path.join(path, '-'.join([name_pma,
                                                          name_cntrl,
                                                          t_type,
                                                          'metadata.qza'])))
        